In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv("E:\dataset\dblp-ref\data3.csv")

# Define the condition to filter rows for removal
condition = (data['references'].notnull() & (data['venue'].notnull() | data['abstract'].notnull())) | (data[['references', 'venue', 'abstract']].notnull().all(axis=1))

data = data[condition]
paper_id_encoder = LabelEncoder()

# Fit and transform the data to obtain numerical values
data['paper_id_encoded'] = paper_id_encoder.fit_transform(data['id'])
encoded_data = pd.DataFrame({'id': data['id'], 'paper_id_encoded': data['paper_id_encoded']})
# Access the encoded values and print them alongside the actual paper_id
#print(encoded_data)
# Create a mapping from paper_id to its encoded value
paper_id_mapping = dict(zip(data['id'], data['paper_id_encoded']))
# Initialize encoded_references1 list outside the outer loop
print(paper_id_mapping)
encoded_paper_ids = []  # List of encoded paper IDs
references = []  # List of references for each paper

# Iterate through the 'references' column for each paper
for paper_id, references_str in zip(data['id'], data['references']):
    reference_ids = references_str.split(',')
    encoded_references = []
    for reference_id in reference_ids:
        reference_id = reference_id.strip()  # Remove leading/trailing spaces
        if reference_id in paper_id_mapping:
            encoded_reference_id = paper_id_mapping[reference_id]
            encoded_references.append(encoded_reference_id)
    if encoded_references:
        encoded_paper_id = paper_id_mapping[paper_id]
        encoded_paper_ids.append(encoded_paper_id)
        references.append(encoded_references)

# Print the result with encoded values
#print(len(encoded_paper_ids), len(references))
result = [encoded_paper_ids, references]
edges = result
print(edges)

        # Modify the edges based on your edge features
source_nodes = edges[0]
target_nodes = edges[1]

        # Create a dictionary to map source nodes to their lists of target nodes
source_to_targets = {}
for source, target in zip(source_nodes, target_nodes):
    if source not in source_to_targets:
         source_to_targets[source] = []
    source_to_targets[source].extend(target)

mapped_source_nodes = []
mapped_target_nodes = []
print(len(source_to_targets.keys()))
        # Iterate through source nodes and their associated target nodes
for source_node, target_nodes in source_to_targets.items():
     mapped_source_nodes.extend([source_node] * len(target_nodes))
     mapped_target_nodes.extend(target_nodes)


print(len(mapped_source_nodes))
print(len(mapped_target_nodes))
result1=[mapped_source_nodes,mapped_target_nodes]
print(result1)




{'4ab3735c-80f1-472d-b953-fa0557fed28b': 0, '4ab39729-af77-46f7-a662-16984fb9c1db': 1, '4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de': 2, '4ab3a98c-3620-47ec-b578-884ecf4a6206': 3, '4ab3b585-82b4-4207-91dd-b6bce7e27c4e': 4, '4ab3e768-78c9-4497-8b8e-9e934cb5f2e4': 5, '4ab3f7cd-140b-4e29-99d4-f4e8006c4f65': 6, '4ab404e2-6f4b-4fb4-b093-50775e765b13': 7, '4ab4244d-fb3e-49a3-b125-367df3d8e6ba': 8, '4ab439a4-9379-44f5-b98b-87125ae7366e': 9, '4ab459ce-5f18-4cc0-9627-1dc557eb8b68': 10, '4ab46288-3a29-471f-95c9-9926df1f93ce': 11, '4ab4c0a1-3c5a-44c6-bdd4-3a0618d303ca': 12, '4ab4c17b-a9ac-4f37-8e71-5977d5db5a07': 13, '4ab4e2a8-18c5-47a0-a295-a63dc68a49ab': 14, '4ab503e2-d7b4-441e-8e0e-edf1b9bfb602': 15, '4ab50bd7-8e59-4974-a6d0-59a577dcc10d': 16, '4ab50fd5-e0e8-44aa-acf1-a7a131293e7c': 17, '4ab54610-4947-46c7-bd56-fc672dfba5cc': 18, '4ab5585e-6a6a-4c92-b37c-e4dbb67fe72e': 19, '4ab572c2-c224-4d2e-832b-8bd440ed4da1': 20, '4ab57ebc-8bdf-4f3b-91c8-c20756fce04a': 21, '4ab59e6c-fa3b-4529-bc39-00da61e0afc4': 2

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences 

data = pd.read_csv("E:\dataset\dblp-ref\data3.csv")

# Define the condition to filter rows for removal
condition = (data['references'].notnull() & (data['venue'].notnull() | data['abstract'].notnull())) | (data[['references', 'venue', 'abstract']].notnull().all(axis=1))

data = data[condition]

author_label_encoder = LabelEncoder()

# Extract the 'title' column from the DataFrame
titles = data['title']

# Tokenize the titles
tokenized_titles = [word_tokenize(title) for title in titles]

# Train a Word2Vec model
model = Word2Vec(tokenized_titles, vector_size=100, window=5, min_count=1, sg=0)

# Get the Word2Vec embeddings for each title
title_embeddings = [model.wv[words] for words in tokenized_titles]

# Determine the maximum sequence length (number of words in the title)
max_seq_length = max(embedding.shape[0] for embedding in title_embeddings)

# Pad the title embeddings to have the same shape (padding with zeros)
title_embeddings_padded = pad_sequences(title_embeddings, dtype='float32', padding='post', maxlen=max_seq_length)

# Convert the padded title embeddings to a NumPy array
title_embeddings_array = np.array(title_embeddings_padded)
paper_id_encoder = LabelEncoder()
# Convert label-encoded author and venue to NumPy arrays
data['encoded_author'] = author_label_encoder.fit_transform(data['authors'])
encoded_author_array = data['encoded_author'].values.reshape(-1, 1)
n_citations_array = data['n_citation'].values.reshape(-1, 1)
year_array = data['year'].values.reshape(-1, 1)

# Create a list to store individual node features
node_features_list = []
data['paper_id_encoded'] = paper_id_encoder.fit_transform(data['id'])
encoded_data = pd.DataFrame({'id': data['id'], 'paper_id_encoded': data['paper_id_encoded']})
paper_id_mapping = dict(zip(data['id'], data['paper_id_encoded']))
print(paper_id_mapping)
# Iterate through unique paper IDs and organize features for each node
unique_paper_ids = data['id'].unique()
for paper_id in unique_paper_ids:
    # Filter data for the current paper_id
    paper_data = data[data['id'] == paper_id]
    # Extract features for the current paper_id
    encoded_author = paper_data['encoded_author'].values[0]
    n_citations = paper_data['n_citation'].values[0]
    year = paper_data['year'].values[0]
    encoded_id = encoded_data.loc[encoded_data['id'] == paper_id, 'paper_id_encoded'].values[0]
    # Organize features for the current node
    node_features = np.array([encoded_id,encoded_author, n_citations, year])
    
    # Append features to the list
    node_features_list.append(node_features)
print(node_features_list)
# Convert the list of node features to a NumPy array
node_features_array = np.array(node_features_list)
print(node_features_array)


title_embeddings_array = title_embeddings_array.astype(np.float32)


{'4ab3735c-80f1-472d-b953-fa0557fed28b': 0, '4ab39729-af77-46f7-a662-16984fb9c1db': 1, '4ab3a4cf-1d96-4ce5-ab6f-b3e19fc260de': 2, '4ab3a98c-3620-47ec-b578-884ecf4a6206': 3, '4ab3b585-82b4-4207-91dd-b6bce7e27c4e': 4, '4ab3e768-78c9-4497-8b8e-9e934cb5f2e4': 5, '4ab3f7cd-140b-4e29-99d4-f4e8006c4f65': 6, '4ab404e2-6f4b-4fb4-b093-50775e765b13': 7, '4ab4244d-fb3e-49a3-b125-367df3d8e6ba': 8, '4ab439a4-9379-44f5-b98b-87125ae7366e': 9, '4ab459ce-5f18-4cc0-9627-1dc557eb8b68': 10, '4ab46288-3a29-471f-95c9-9926df1f93ce': 11, '4ab4c0a1-3c5a-44c6-bdd4-3a0618d303ca': 12, '4ab4c17b-a9ac-4f37-8e71-5977d5db5a07': 13, '4ab4e2a8-18c5-47a0-a295-a63dc68a49ab': 14, '4ab503e2-d7b4-441e-8e0e-edf1b9bfb602': 15, '4ab50bd7-8e59-4974-a6d0-59a577dcc10d': 16, '4ab50fd5-e0e8-44aa-acf1-a7a131293e7c': 17, '4ab54610-4947-46c7-bd56-fc672dfba5cc': 18, '4ab5585e-6a6a-4c92-b37c-e4dbb67fe72e': 19, '4ab572c2-c224-4d2e-832b-8bd440ed4da1': 20, '4ab57ebc-8bdf-4f3b-91c8-c20756fce04a': 21, '4ab59e6c-fa3b-4529-bc39-00da61e0afc4': 2

In [4]:
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 300
batch_size = 256

In [5]:
def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))

    return keras.Sequential(fnn_layers, name=name)

In [6]:
edges = result1
# Create a node features array of shape [num_nodes, num_features].
print(edges[0])
print(edges[1])
node_features=node_features_array
title_features=title_embeddings_array

# Create graph info tuple with node_features, edges, and edge_weights.
graph_info = (node_features,title_features, edges)

print("Edges shape:", len(edges[0]),len(edges[1]))
print("Nodes shape:", node_features.shape)

[12, 24, 30, 37, 42, 55, 82, 84, 114, 129, 131, 144, 149, 158, 196, 197, 205, 216, 245, 248, 262, 262, 285, 298, 303, 312, 322, 328, 328, 330, 345, 382, 389, 409, 409, 454, 489, 499, 503, 511, 514, 527, 541, 544, 575, 576, 591, 606, 618, 618, 633, 639, 685, 685, 704, 719, 734, 788, 789, 793, 793, 794, 809, 882, 899, 905, 905, 905, 977, 977, 1000, 1005, 1008, 1016, 1047, 1075, 1084, 1120, 1127, 1159, 1186, 1186, 1186, 1188, 1189, 1196, 1207, 1231, 1255, 1277, 1277, 1283, 1288, 1306, 1325, 1355, 1359, 1386, 1401, 1407, 1413, 1419, 1420, 1426, 1447, 1460, 1473, 1491, 1511, 1516, 1529, 1542, 1551, 1582, 1594, 1617, 1641, 1642, 1660, 1678, 1704, 1717, 1718, 1744, 1746, 1750, 1750, 1795, 1811, 1811, 1812, 1822, 1842, 1846, 1891, 1892, 1900, 1906, 1909, 1916, 1923, 1931, 1938, 1951, 1951, 1956, 1987, 1998, 2001, 2004, 2008, 2050, 2057, 2092, 2093, 2107, 2114, 2132, 2146, 2167, 2172, 2189, 2211, 2225, 2256, 2304, 2311, 2321, 2327, 2339, 2362, 2371, 2379, 2384, 2392, 2400, 2423, 2437, 2449, 245

In [7]:
class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.2,
        aggregation_type="mean",
        combination_type="concat",
        normalize=False,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        if self.combination_type == "gated":
            self.update_fn = layers.GRU(
                units=hidden_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                dropout=dropout_rate,
                return_state=True,
                recurrent_dropout=dropout_rate,
            )
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_repesentations):
        # node_indices shape is [num_edges].
        # neighbour_messages shape: [num_edges, representation_dim].
        # node_repesentations shape is [num_nodes, representation_dim].
        num_nodes = node_repesentations.shape[0]
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")
 
        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        # node_repesentations shape is [num_nodes, representation_dim].
        # aggregated_messages shape is [num_nodes, representation_dim].
        if self.combination_type == "gru":
            # Create a sequence of two elements for the GRU layer.
            h = tf.stack([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            # Concatenate the node_repesentations and aggregated_messages.
            h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            # Add node_repesentations and aggregated_messages.
            h = node_repesentations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        # Apply the processing function.
        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):
        """Process the inputs to produce the node_embeddings.

        inputs: a tuple of three elements: node_repesentations, edges, edge_weights.
        Returns: node_embeddings of shape [num_nodes, representation_dim].
        """

        node_repesentations, edges = inputs

        # Modify the edges based on your edge features
        source_nodes = edges[0]
        target_nodes = edges[1]

       
        # Get neighbour representations based on the mapped target nodes
        neighbour_repesentations = tf.gather(node_repesentations, target_nodes)

        # Prepare the messages of the neighbors.
        neighbour_messages = self.prepare(neighbour_repesentations)
        # Aggregate the neighbour messages.
        aggregated_messages = self.aggregate(
            source_nodes, neighbour_messages, node_repesentations
        )
        print(aggregated_messages[4408])
        # Update the node embedding with the neighbour messages.
        return self.update(node_repesentations, aggregated_messages)


In [8]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        node_features, title_features, edges, = graph_info
        self.node_features = node_features
        self.title_features = title_features
        self.edges = edges
       

        # Create a process layer for the first feature branch.
        self.preprocess_node = create_ffn(hidden_units, dropout_rate, name="preprocess_node")
        # Create a process layer for the second feature branch.
        self.preprocess_title = create_ffn(hidden_units, dropout_rate, name="preprocess_title")

        # Create the first GraphConv layer for the first branch.
        self.conv1_node = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1_node",
        )
        # Create the first GraphConv layer for the second branch.
        self.conv1_title = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1_title",
        )

        # Create the second GraphConv layers for both branches similarly
        self.conv2_node = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2_node",
        )
        self.conv2_title = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2_title",
        )

        # Create a postprocess layer for the first feature branch.
        self.postprocess_node = create_ffn(hidden_units, dropout_rate, name="postprocess_node")
        # Create a postprocess layer for the second feature branch.
        self.postprocess_title = create_ffn(hidden_units, dropout_rate, name="postprocess_title")

    def call(self, input_node_indices, input_title_indices):
        # Preprocess the node_features for both branches.
        x_node = self.preprocess_node(self.node_features)
        x_title = self.preprocess_title(self.title_features)

        # Apply the first GraphConv layers for both branches.
        x1_node = self.conv1_node((x_node, self.edges))
        x1_title = self.conv1_title((x_title, self.edges))

        # Apply the second GraphConv layers for both branches.
        x2_node = self.conv2_node((x1_node, self.edges))
        x2_title = self.conv2_title((x1_title, self.edges))

        # Postprocess node embeddings for both branches.
        x_node = self.postprocess_node(x2_node)
        x_title = self.postprocess_title(x2_title)

        # Fetch node embeddings for the input node_indices for both branches.
        node_embeddings_node = tf.gather(x_node, input_node_indices)
        node_embeddings_title = tf.gather(x_title, input_title_indices)

        return x_node, x_title


In [9]:
gnn_model = GNNNodeClassifier(
    graph_info=graph_info,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

node_indices = edges[0]  # Replace with your actual node indices
title_indices = edges[0]  # Replace with your actual title indices

# Call the GNN model with input indices
node_embeddings, title_embeddings = gnn_model(node_indices, title_indices)

# Print the embeddings separately
print("Node Embeddings:")
print(node_embeddings.shape)

print("\nTitle Embeddings:")
print(title_embeddings.shape)

tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.], shape=(32,), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [10]:
node_embeddings_np = node_embeddings.numpy()
title_embeddings_np = title_embeddings.numpy()

# Now node_embeddings_np and title_embeddings_np are NumPy arrays
print("Node Embeddings (NumPy):")
print(node_embeddings_np)

print("\nTitle Embeddings (NumPy):")
print(title_embeddings_np)

Node Embeddings (NumPy):
[[ 0.03536864 -0.04857356 -0.06369511 ... -0.01145632 -0.03129853
   0.13299702]
 [ 0.01192929 -0.02766804 -0.04667008 ... -0.03254981  0.01251745
   0.08494731]
 [-0.00766166  0.00817214 -0.03587171 ... -0.05092892  0.02910675
   0.03998363]
 ...
 [-0.03171673  0.02143175 -0.00956502 ... -0.07638484  0.01545429
  -0.01234641]
 [-0.02293938  0.03671679  0.00282553 ... -0.08158378  0.01407265
  -0.00470996]
 [-0.0337098   0.01817174 -0.01513678 ... -0.07348011  0.01756514
  -0.01250642]]

Title Embeddings (NumPy):
[[[-0.01176498  0.02644468 -0.0236432  ...  0.02207712  0.02465005
   -0.07037087]
  [-0.02231221  0.02058514 -0.01782993 ...  0.03021483  0.03406532
   -0.0524041 ]
  [-0.01984436  0.02079791 -0.02942257 ...  0.02930935  0.03685765
   -0.05484469]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.     

In [11]:
class InnerProductDecoder(tf.keras.layers.Layer):
    """Decoder model layer for link prediction."""
    def __init__(self, input_dim, dropout=0., act=tf.nn.sigmoid, flatten=True, **kwargs):
        super(InnerProductDecoder, self).__init__(**kwargs)
        self.dropout = dropout
        self.act = act
        self.flatten = flatten

    def call(self, inputs):
        inputs = tf.nn.dropout(inputs, 1 - self.dropout)
        x = tf.transpose(inputs)
        x = tf.matmul(inputs, x)
        if self.flatten:
            x = tf.reshape(x, [-1])
        outputs = self.act(x)
        return outputs

class SeparateLinkPredictionModel(tf.keras.Model):
    def __init__(self, embedding_dim, dropout_rate=0.2, **kwargs):
        super(SeparateLinkPredictionModel, self).__init__(**kwargs)
        
        self.dropout_rate = dropout_rate
        self.flatten_layer = layers.Flatten()
        self.node_decoder = InnerProductDecoder(
            input_dim=embedding_dim,
            dropout=dropout_rate,
            act=tf.nn.sigmoid,
            flatten=True,
            name="node_link_prediction_decoder",
        )
        self.title_decoder = InnerProductDecoder(
            input_dim=embedding_dim,
            dropout=dropout_rate,
            act=tf.nn.sigmoid,
            flatten=True,
            name="title_link_prediction_decoder",
        )

    def call(self, node_embeddings, title_embeddings):
        # Apply dropout to node and title embeddings
        node_embeddings = tf.nn.dropout(node_embeddings, 1 - self.dropout_rate)
        title_embeddings = tf.nn.dropout(title_embeddings, 1 - self.dropout_rate)

        # Flatten the embeddings
        node_flat = self.flatten_layer(node_embeddings)
        title_flat = self.flatten_layer(title_embeddings)

        # Predict links using InnerProductDecoder
        node_link_predictions = self.node_decoder(node_flat)
        title_link_predictions = self.title_decoder(title_flat)

        return node_link_predictions, title_link_predictions

# Assuming you have obtained node_embeddings and title_embeddings
embedding_dim = node_embeddings.shape[-1]

# Create the separate link prediction model
separate_link_prediction_model = SeparateLinkPredictionModel(
    embedding_dim=embedding_dim,
    dropout_rate=dropout_rate,
    name="separate_link_prediction_model",
)

# Call the model with node and title embeddings
node_link_predictions, title_link_predictions = separate_link_prediction_model(node_embeddings, title_embeddings)

# Print link predictions separately
print("Node Link Predictions:")
print(len(node_link_predictions))
print(node_link_predictions)

print("\nTitle Link Predictions:")
print(title_link_predictions.shape)
print(title_link_predictions)


Node Link Predictions:
69973225
tf.Tensor([0.58568645 0.49856156 0.504446   ... 0.5109249  0.50470984 0.52637285], shape=(69973225,), dtype=float32)

Title Link Predictions:
(69973225,)
tf.Tensor([0.86034274 0.62554055 0.5529529  ... 0.58447015 0.5289043  0.80663246], shape=(69973225,), dtype=float32)


In [12]:
node_link_predictions_matrix = np.reshape(node_link_predictions, (int(np.sqrt(len(node_link_predictions))), -1))
print(node_link_predictions_matrix.shape)
# Extracting source and target nodes for node link predictions
node_source_nodes, node_target_nodes = np.nonzero(node_link_predictions_matrix)
print(node_source_nodes[0])
# Extracting probabilities for the predicted links
node_predicted_probabilities = node_link_predictions_matrix[node_source_nodes, node_target_nodes]

# Creating a list of predicted links with their probabilities for node link predictions
node_predicted_links = list(zip(node_source_nodes, node_target_nodes, node_predicted_probabilities))

# Printing the list of predicted links for node link predictions
print("Node Link Predictions:")

print(node_predicted_links[0])


(8365, 8365)
0
Node Link Predictions:
(0, 0, 0.58568645)


In [13]:
print(edges[0])
print(edges[1])

ListWrapper([12, 24, 30, 37, 42, 55, 82, 84, 114, 129, 131, 144, 149, 158, 196, 197, 205, 216, 245, 248, 262, 262, 285, 298, 303, 312, 322, 328, 328, 330, 345, 382, 389, 409, 409, 454, 489, 499, 503, 511, 514, 527, 541, 544, 575, 576, 591, 606, 618, 618, 633, 639, 685, 685, 704, 719, 734, 788, 789, 793, 793, 794, 809, 882, 899, 905, 905, 905, 977, 977, 1000, 1005, 1008, 1016, 1047, 1075, 1084, 1120, 1127, 1159, 1186, 1186, 1186, 1188, 1189, 1196, 1207, 1231, 1255, 1277, 1277, 1283, 1288, 1306, 1325, 1355, 1359, 1386, 1401, 1407, 1413, 1419, 1420, 1426, 1447, 1460, 1473, 1491, 1511, 1516, 1529, 1542, 1551, 1582, 1594, 1617, 1641, 1642, 1660, 1678, 1704, 1717, 1718, 1744, 1746, 1750, 1750, 1795, 1811, 1811, 1812, 1822, 1842, 1846, 1891, 1892, 1900, 1906, 1909, 1916, 1923, 1931, 1938, 1951, 1951, 1956, 1987, 1998, 2001, 2004, 2008, 2050, 2057, 2092, 2093, 2107, 2114, 2132, 2146, 2167, 2172, 2189, 2211, 2225, 2256, 2304, 2311, 2321, 2327, 2339, 2362, 2371, 2379, 2384, 2392, 2400, 2423, 243

In [12]:
# Assuming node_link_predictions_matrix is a 2D numpy array containing link predictions

# Extracting predictions only for node 12
node_12_predictions = node_link_predictions_matrix[,:]
print(type(node_12_predictions))
# Find the index of the maximum probability
max_index = np.argmax(node_12_predictions)

# Get the maximum probability
max_probability = node_12_predictions[max_index]
print(f"Index with the highest link probability: {max_index}")
print(f"Highest link probability: {max_probability}")

SyntaxError: invalid syntax (2452568212.py, line 4)

ListWrapper([12, 24, 30, 37, 42, 55, 82, 84, 114, 129, 131, 144, 149, 158, 196, 197, 205, 216, 245, 248, 262, 262, 285, 298, 303, 312, 322, 328, 328, 330, 345, 382, 389, 409, 409, 454, 489, 499, 503, 511, 514, 527, 541, 544, 575, 576, 591, 606, 618, 618, 633, 639, 685, 685, 704, 719, 734, 788, 789, 793, 793, 794, 809, 882, 899, 905, 905, 905, 977, 977, 1000, 1005, 1008, 1016, 1047, 1075, 1084, 1120, 1127, 1159, 1186, 1186, 1186, 1188, 1189, 1196, 1207, 1231, 1255, 1277, 1277, 1283, 1288, 1306, 1325, 1355, 1359, 1386, 1401, 1407, 1413, 1419, 1420, 1426, 1447, 1460, 1473, 1491, 1511, 1516, 1529, 1542, 1551, 1582, 1594, 1617, 1641, 1642, 1660, 1678, 1704, 1717, 1718, 1744, 1746, 1750, 1750, 1795, 1811, 1811, 1812, 1822, 1842, 1846, 1891, 1892, 1900, 1906, 1909, 1916, 1923, 1931, 1938, 1951, 1951, 1956, 1987, 1998, 2001, 2004, 2008, 2050, 2057, 2092, 2093, 2107, 2114, 2132, 2146, 2167, 2172, 2189, 2211, 2225, 2256, 2304, 2311, 2321, 2327, 2339, 2362, 2371, 2379, 2384, 2392, 2400, 2423, 243

In [29]:
import numpy as np
#print(len(edges[0]))
#print(len(edges[1]))
import numpy as np

total_nodes = 8365

# Assuming node_embeddings is a numpy array containing node embeddings
# Assuming edges[0] and edges[1] are lists containing source and target nodes
edges1 = list(edges[0])
edges2 = list(edges[1])

# Initialize bootstrapped_edge_embeddings as an empty list
import numpy as np

total_nodes = 8365

# Assuming node_embeddings is a numpy array containing node embeddings
# Assuming edges[0] and edges[1] are lists containing source and target nodes
edges1 = list(edges[0])
edges2 = list(edges[1])

# Initialize bootstrapped_edge_embeddings as an empty list
bootstrapped_edge_embeddings = []

# Extracting node embeddings for source nodes
for (src, tgt) in zip(edges[0], edges[1]):
    source_node_embedding = node_embeddings[src]
    target_node_embedding = node_embeddings[tgt]
    edge_embedding = np.multiply(source_node_embedding, target_node_embedding)
    bootstrapped_edge_embeddings.append(edge_embedding)

# Convert the list to a NumPy array
bootstrapped_edge_embeddings = np.array(bootstrapped_edge_embeddings)
print(bootstrapped_edge_embeddings[1])
# Now, for nodes with multiple edges, aggregate the Hadamard products (e.g., by summing)
unique_nodes = set(edges[0])
aggregated_embeddings = []  # Create a new list to store aggregated embeddings

for node in unique_nodes:
    node_indices = [idx for idx, src in enumerate(edges[0]) if src == node]
    aggregated_embedding = np.sum(bootstrapped_edge_embeddings[node_indices, :], axis=0)
    aggregated_embeddings.append(aggregated_embedding)

# Convert the aggregated embeddings list to a NumPy array
aggregated_embeddings = np.array(aggregated_embeddings)
print(len(aggregated_embeddings))
print(aggregated_embeddings[1])
non_edge_nodes = list(set(range(total_nodes)) - set(edges[0]))
non_edge_node_embeddings = [node_embeddings[i] for i in non_edge_nodes]
print(len(non_edge_node_embeddings))
# Including node embeddings for other nodes where edges are not present
bootstrapped_edge_embeddings = np.concatenate((aggregated_embeddings, non_edge_node_embeddings), axis=0)
print(len(bootstrapped_edge_embeddings))

[ 2.11390638e-04 -1.12171474e-04  6.77625090e-03  8.57962959e-07
  1.16148181e-02  2.87749479e-03  3.84138024e-04  7.41572701e-04
  1.55787251e-03 -4.53281200e-06 -3.71716043e-04  3.18480679e-03
  3.10646859e-03  8.41984802e-05  9.43399573e-05  1.29100808e-03
 -7.84781369e-05 -1.34242029e-04  9.74593044e-04  7.66634615e-03
  4.76145605e-03  7.71317165e-03  6.67474815e-04  1.35502238e-02
 -2.10466984e-04  3.79222096e-03  7.71717168e-04  2.03814925e-04
  6.91931508e-03  9.71733418e-04  1.34681049e-03  2.97580194e-03]
479
[ 1.80294111e-04 -7.17860021e-05  7.89900124e-03  3.23625027e-05
  6.15898706e-03  1.22048082e-02 -1.14767929e-03 -3.51106370e-04
 -4.85086057e-04 -6.17102953e-04 -4.92211955e-04  1.74375693e-03
  1.94048378e-04 -5.68317737e-05  9.90467262e-04  1.40997197e-03
 -1.05589890e-04 -7.10567401e-05  2.20460212e-03  1.47257745e-02
  4.35396284e-03  9.45805106e-03  3.71012604e-03  1.91166177e-02
 -1.37680455e-03  3.31171718e-03  3.31654772e-03  2.87962169e-03
  8.96533299e-03 -2.

In [11]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming edges[0] and edges[1] are arrays representing directed edges
# Assuming total_nodes is the total number of nodes in your network
total_nodes = 8365

# Create an adjacency matrix for a directed graph
adjacency_matrix = np.zeros((total_nodes, total_nodes))

# Assuming you have a binary indicator for the presence of a directed edge
for src, tgt in zip(edges[0], edges[1]):
    adjacency_matrix[src, tgt] = 1

# Assuming you have node_embeddings from the previous code
# Make sure node_embeddings is a NumPy array
node_embeddings = np.array(node_embeddings)

# Initialize a 2D array to store combined embeddings
combined_embeddings_2d = np.zeros((total_nodes, total_nodes),dtype=object)

# Hadamard product for each edge and update combined_embeddings_2d
for src, tgt in zip(edges[0], edges[1]):
    edge_emb = np.multiply(node_embeddings[src], node_embeddings[tgt])
    combined_embeddings_2d[src, tgt] = edge_emb
print(combined_embeddings_2d[12][4408])

[ 4.0168961e-04 -2.3912638e-03  4.4391099e-03  1.4642184e-03
  1.3948072e-03  1.3204062e-03  5.5179198e-04 -5.1620236e-04
  7.8703320e-05 -3.3945729e-05  2.2243518e-04  1.4163171e-03
  2.7203371e-04 -5.9512869e-04  2.6476491e-05  5.2981515e-04
 -4.5195391e-04 -4.2307028e-04 -1.2455445e-03  6.5550505e-04
  5.9976912e-04  2.0986768e-04  4.9757888e-04 -1.0195093e-03
  5.7352758e-05 -1.0561712e-04  1.1298702e-03  1.2173493e-03
  1.0943562e-03  1.9176904e-03 -9.1541306e-06 -4.4840202e-04]


In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming edges[0] and edges[1] are arrays representing directed edges
# Assuming total_nodes is the total number of nodes in your network
total_nodes = 8365

# Create an adjacency matrix for a directed graph
adjacency_matrix = np.zeros((total_nodes, total_nodes))

# Assuming you have a binary indicator for the presence of a directed edge
for src, tgt in zip(edges[0], edges[1]):
    adjacency_matrix[src, tgt] = 1

# Assuming you have node_embeddings from the previous code
# Make sure node_embeddings is a NumPy array
node_embeddings = np.array(node_embeddings)

# Initialize a 2D array to store combined embeddings
combined_embeddings_2d = np.zeros((total_nodes, total_nodes), dtype=object)

# Iterate through each row in the adjacency matrix
for src in range(total_nodes):
    for tgt in range(total_nodes):
        if adjacency_matrix[src, tgt] == 0:
            # If no edge, store the source node embedding in combined_embeddings_2d
            combined_embeddings_2d[src, tgt] = node_embeddings[src]
        else:
            # If edge, store the Hadamard product in combined_embeddings_2d
            edge_emb = np.multiply(node_embeddings[src], node_embeddings[tgt])
            combined_embeddings_2d[src, tgt] = edge_emb




In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X = combined_embeddings_2d.reshape(total_nodes, -1)
print(X.shape)


labels_exist = adjacency_matrix
print(labels_exist.shape)
X_train, X_test, y_train, y_test = train_test_split(X, labels_exist, test_size=0.2, random_state=42)

classifier = LogisticRegression()
classifier.fit(X_train.tolist(), y_train)  # Convert X_train to list for compatibility with Logistic Regression

# Make predictions on the test set
predictions = classifier.predict(X_test.tolist())  # Convert X_test to list for compatibility with Logistic Regression

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


(8365, 8365)
(8365, 8365)


ValueError: Found array with dim 3. LogisticRegression expected <= 2.

In [21]:
import dgl
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Assuming combined_embeddings_2d is a list of NumPy arrays representing node embeddings
# Assuming adjacency_matrix is your adjacency matrix
combined_embeddings_2d = [np.array(embedding) for embedding in combined_embeddings_2d]
labels_exist = adjacency_matrix

# Find the maximum length of node embeddings
max_length = max(len(embedding) for embedding in combined_embeddings_2d)

# Pad or truncate each embedding to the maximum length
padded_embeddings = [np.pad(embedding, (0, max_length - len(embedding)), 'constant') for embedding in combined_embeddings_2d]

# Create a graph from your adjacency matrix using DGL
g = dgl.graph((labels_exist.nonzero()[0], labels_exist.nonzero()[1]))

# Split the data into training and testing sets
train_idx, test_idx = train_test_split(range(g.number_of_nodes()), test_size=0.2, random_state=42)
labels_exist_train = labels_exist[train_idx, :][:, train_idx]
labels_exist_test = labels_exist[test_idx, :][:, test_idx]

# Convert node embeddings to PyTorch tensors
node_embeddings = [torch.tensor(embedding, dtype=torch.float32) for embedding in padded_embeddings]

# Create a tensor for the adjacency matrix
labels_exist_train = torch.tensor(labels_exist_train, dtype=torch.float32)
labels_exist_test = torch.tensor(labels_exist_test, dtype=torch.float32)

# Define a simple GNN model for binary classification
class GNNBinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GNNBinaryClassifier, self).__init__()
        self.conv1 = dgl.nn.GraphConv(input_dim, hidden_dim)
        self.conv2 = dgl.nn.GraphConv(hidden_dim, 1)  # Output 1 dimension for binary classification

    def forward(self, g, features):
        x = torch.relu(self.conv1(g, features))
        x = self.conv2(g, x)
        return x

# Initialize the model, loss function, and optimizer
model = GNNBinaryClassifier(input_dim=node_embeddings[0].shape[1], hidden_dim=64)
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(50):
    model.train()
    logits = model(g, node_embeddings)
    loss = criterion(logits.view(-1), labels_exist_train.view(-1))  # Flatten logits and labels
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    logits_test = model(g, node_embeddings)
    predictions = (torch.sigmoid(logits_test) > 0.5).int()  # Convert logits to binary predictions
    accuracy = accuracy_score(labels_exist_test.view(-1).numpy(), predictions.view(-1).numpy())
    print(f"Accuracy: {accuracy}")


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.